<a href="https://colab.research.google.com/github/Alexx683/GitHubTestProjects/blob/main/TranscriptionVideoDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're looking at this on GitHub and new to Python Notebooks or Colab, click the Google Colab badge above 👆

#📼 OpenAI Whisper + Google Drive Video Transcription

📺 Getting started video: https://youtu.be/YGpYinji7II

###This application will extract audio from all the video files in a Google Drive folder and create a high-quality transcription with OpenAI's Whisper automatic speech recognition system.

*Note: This requires giving the application permission to connect to your drive. Only you will have access to the contents of your drive, but please read the warnings carefully.*

This notebook application:
1. Connects to your Google Drive when you give it permission.
2. Creates a WhisperVideo folder and three subfolders (ProcessedVideo, AudioFiles and TextFiles.)
3. When you run the application it will search for all the video files (.mp4, .mov, mkv and .avi) in your WhisperVideo folder, transcribe them and then move the file to WhisperVideo/ProcessedVideo and save the transcripts to WhisperVideo/TextFiles. It will also add a copy of the new audio file to WhisperVideo/AudioFiles

###**For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Note: If you add a new file after running this application you'll need to remount the drive in step 1 to make them searchable**

##1. Load the code libraries

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
# model = whisper.load_model("base.en")
model = whisper.load_model("small.en") # load the small model
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

##2. Give the application permission to mount the drive and create the folders

In [2]:
# Create the Drive folders
import os
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)


Mounted at /content/drive


##3. Upload any video files you want transcribed in the "WhisperVideo" folder in your Google Drive.

##4. Extract audio from the video files and create a transcription

In [6]:
!pip install openai-whisper
!pip install librosa soundfile

import librosa
import soundfile as sf
import os
import whisper

# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Initialize Whisper model
model = whisper.load_model("base")  # Replace "base" with "small", "medium", etc., if needed

# Define folders
base_path = "/content/drive/MyDrive/WhisperVideo/"
audio_folder = os.path.join(base_path, "AudioFiles/")
text_folder = os.path.join(base_path, "TextFiles/")
processed_folder = os.path.join(base_path, "ProcessedVideo/")

# Ensure directories exist
os.makedirs(audio_folder, exist_ok=True)
os.makedirs(text_folder, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)

# Get the list of video files
video_files = os.listdir(base_path)

# Loop through the video files and transcribe them
for video_file in video_files:
    if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
        continue

    try:
        # Extract the audio from the video file
        video_path = os.path.join(base_path, video_file)
        audio_path = os.path.join(audio_folder, video_file[:-4] + ".wav")  # Change extension to .wav

        # Load audio from the video file
        y, sr = librosa.load(video_path, sr=16000)
        sf.write(audio_path, y, sr)

        # Transcribe the audio file using Whisper
        result = model.transcribe(audio_path)
        text = result["text"].strip()
        text = text.replace(". ", ".\n\n")  # Format text with double line breaks for readability

        # Save the transcription as a text file
        text_file = video_file[:-4] + ".txt"  # Change extension to .txt
        text_path = os.path.join(text_folder, text_file)
        with open(text_path, "w") as f:
            f.write(text)

        # Move the video file to the ProcessedVideo folder
        processed_path = os.path.join(processed_folder, video_file)
        os.rename(video_path, processed_path)

        # Print progress
        print(f"Processed {video_file} and saved the transcription as {text_file}")

    except Exception as e:
        print(f"Error processing {video_file}: {e}")


Mounted at /content/drive


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 129MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Processed About.mp4 and saved the transcription as About.txt
